# Denoising Diffusion Probabilistic Models

In [42]:
import torch 
import torch.nn as nn

In [43]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride)
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))
    
class DownBlock(nn.Module):
    def __init__(self, filters, in_channels):
        super(DownBlock, self).__init__()
        conv_blocks = [ConvBlock(in_channels, filters[0])]
        for i in range(1, len(filters)):
            conv_blocks.append(ConvBlock(filters[i-1], filters[i]))

        self.conv_blocks = nn.Sequential(*conv_blocks)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        residual_outputs = []
        for conv_block in self.conv_blocks:
            x = conv_block(x)
            residual_outputs.append(x)
            x = self.maxpool(x)

        return residual_outputs, x

class UpBlock(nn.Module):
    def __init__(self, filters):
        super(UpBlock, self).__init__()
        layers = []
        for i in range(len(filters) - 2):
            layers.extend(nn.Sequential(
                ConvBlock(filters[i], filters[i+1])), 
                nn.ConvTranspose2d(filters[i+1], filters[i+1], 2)
            )
        
        layers.append(ConvBlock(filters[-2], filters[-1]))
    
    def forward(self, x, residual_outputs):
        for i in range(len(self.layers)):
            x = torch.cat([x, residual_outputs[-(i+1)]], dim=1)
            x = self.layers[i](x)
        
        return x

In [44]:
db = DownBlock([32, 64, 128], 3)

In [45]:
db

DownBlock(
  (conv_blocks): Sequential(
    (0): ConvBlock(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    )
    (1): ConvBlock(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    )
    (2): ConvBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    )
  )
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [46]:
a = db(torch.randn(1, 3, 128, 128))
a[1].shape

torch.Size([1, 128, 12, 12])

In [47]:
a[1].shape

torch.Size([1, 128, 12, 12])

In [48]:
bottom_conv = ConvBlock(128, 256)
bottom_conv(a[1]).shape

torch.Size([1, 256, 8, 8])

In [49]:
up = UpBlock([256, 128, 64, 32])
up(bottom_conv(a[1]), a[0])

TypeError: list.append() takes exactly one argument (2 given)